In [ ]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
%matplotlib inline
from tqdm import notebook
import tensorflow as tf
from scipy import stats
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings("ignore")

### Simulate data

In [ ]:
np.random.seed(2020)

# generate weibull distribution parameter
shape=np.random.uniform(1,5,1000)
scale=np.random.uniform(0.5,2,1000)


# the full design matrix
x=np.c_[shape,scale]


y=(np.random.weibull(shape,size=1000)*scale).reshape(-1,1)


train_x=x[:700,:]
train_y=y[:700,:]

test_x=x[700:,:]
test_y=y[700:,:]

ntrain=len(train_x)
ntest=len(test_x)


### Collaborating Network(CN)

In [ ]:
def variables_from_scope(scope_name):
    """
    Returns a list of all trainable variables in a given scope. This is useful when
    you'd like to back-propagate only to weights in one part of the network
    (in our case, the generator or the discriminator).
    """
    return tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope=scope_name)

In [ ]:
 # Graph parameters
intermediate_layer_size = 100
intermediate_layer_size2 = 80
intermediate_layer_size3 = 60
# Training parameters
batch_size = 128
pre_iter= 20000
iters= 20000

In [ ]:
# f function learn the inverse of cdf
def f(x,q):
    """x:input feature and treatment,
    q:percentile,
    """
    z0=tf.concat([x,q],axis=1)
    hidden_layer = tf.compat.v1.layers.dense(z0, intermediate_layer_size, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=.001),name="f1", activation=tf.compat.v1.nn.elu,reuse=None)
    hidden_layer_bn = tf.compat.v1.layers.batch_normalization(hidden_layer,name="f1bn")
    hidden_layer2 = tf.compat.v1.layers.dense(hidden_layer_bn, intermediate_layer_size2, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=.001),name="f2", activation=tf.keras.activations.elu,reuse=None)
    hidden_layer2_bn = tf.compat.v1.layers.batch_normalization(hidden_layer2,name="f2bn")
    hidden_layer3 = tf.compat.v1.layers.dense(hidden_layer2_bn, intermediate_layer_size3, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=.001),name="f3", activation=tf.compat.v1.nn.elu,reuse=None)
    hidden_layer3_bn = tf.compat.v1.layers.batch_normalization(hidden_layer2,name="f3bn")
    yq = tf.compat.v1.layers.dense(hidden_layer3_bn,1,kernel_initializer=tf.keras.initializers.RandomNormal(stddev=.001), name="f4", activation=None,reuse=None)
    return yq

In [ ]:
# g function learn the cdf
def g(yq,x):
    """
    yq:quantile:,
    x:input feature and treatment,
    """
    z1=tf.concat([yq,x],axis=1)
    hidden_layer = tf.compat.v1.layers.dense(z1, intermediate_layer_size,kernel_initializer=tf.compat.v1.initializers.random_normal(stddev=.001), name="g1", activation=tf.compat.v1.nn.elu,reuse=None)
    hidden_layer_bn = tf.compat.v1.layers.batch_normalization(hidden_layer,name="g1bn")
    hidden_layer2 = tf.compat.v1.layers.dense(hidden_layer_bn, intermediate_layer_size2, kernel_initializer=tf.compat.v1.initializers.random_normal(stddev=.001),name="g2", activation=tf.compat.v1.nn.elu,reuse=None)
    hidden_layer2_bn = tf.compat.v1.layers.batch_normalization(hidden_layer2,name="g2bn")
    gq_logit = tf.compat.v1.layers.dense(hidden_layer2_bn, 1,kernel_initializer=tf.initializers.glorot_normal, name="g3", activation=None,reuse=None)
    gq_logit_bn=tf.keras.layers.BatchNormalization(axis=-1,momentum=.1,trainable=False)(gq_logit)*tf.math.sqrt(3.29)
    return gq_logit_bn

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:

tf.compat.v1.reset_default_graph()

# Placeholders
y_ = tf.compat.v1.placeholder(tf.float32, [None, 1])

pre_y= tf.compat.v1.placeholder(tf.float32, [None, 1])
x_=tf.compat.v1.placeholder(tf.float32, [None, x.shape[1]])
q_ = tf.compat.v1.placeholder(tf.float32, [None, 1])


# f
with tf.compat.v1.variable_scope("f") as scope:
    yq_= f(x_,q_)


ylessthan= tf.cast(tf.less_equal(y_,yq_),tf.float32)
ylessthan_pre= tf.cast(tf.less_equal(y_,pre_y),tf.float32)


with tf.compat.v1.variable_scope("g") as scope:
    gq_logit = g(yq_,x_)
    gq=tf.sigmoid(gq_logit)*.99999+.00001
    scope.reuse_variables()
    gq_logit_pre = g(pre_y,x_)


#Losses
f_loss = tf.reduce_mean(tf.square(q_-gq))
g_loss = 1*tf.compat.v1.losses.sigmoid_cross_entropy(ylessthan,gq_logit)
# +\    .05*tf.compat.v1.losses.mean_squared_error(q_,gq)

#pre-loss
g_loss_pre = tf.compat.v1.losses.sigmoid_cross_entropy(ylessthan_pre,gq_logit_pre)

# Optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4)
pre_step= optimizer.minimize(g_loss_pre,var_list=variables_from_scope("g"))
g_step = optimizer.minimize(g_loss, var_list=variables_from_scope("g"))
f_step = optimizer.minimize(f_loss, var_list=variables_from_scope("f"))

# Initializer
initialize_all = tf.compat.v1.global_variables_initializer()

### Single Evaluation

In [ ]:
sess = tf.compat.v1.Session()
sess.run(initialize_all)
gl=[]
fl=[]


for k in notebook.tnrange(pre_iter):
    i=np.random.choice(ntrain,batch_size,replace=False)
    ytmp = train_y[i,:]
    xtmp= train_x[i,:]
    #when we do not have f initially, we use a uniform distribution to extract points from support
    pre_ytmp=np.random.uniform(-1,14,(batch_size,1))  
    sess.run(pre_step,feed_dict={y_: ytmp,
                                x_:xtmp,
                                pre_y:pre_ytmp})
    

for t in notebook.tnrange(iters):
    #randomly generate a minibatch from valid set
    i=np.random.choice(ntrain,batch_size,replace=False)
    qtmp=np.random.uniform(0, 1.0, [batch_size, 1])
    ytmp = train_y[i,:]
    xtmp= train_x[i,:]
    
    for i in range(0,1):
        _,glt=sess.run([g_step,g_loss], feed_dict={y_: ytmp,
                                x_:xtmp,
                                q_:qtmp})
    gl.append(glt)
    
    for i in range(0,2):
        _,flt=sess.run([f_step,f_loss],feed_dict={y_: ytmp,
                                x_:xtmp,
                                q_:qtmp})
    fl.append(flt)

    

#### P(Y>1|x)

In [ ]:
#true
tsuv1=1-stats.weibull_min.cdf(1,c=test_x[:,0],scale=test_x[:,1])


#cdf estimate by g
gsuv1=1.-sess.run(gq ,feed_dict={x_:test_x,
                                yq_:np.repeat(1,len(test_x)).reshape(-1,1),
                                }).ravel()



# estimation by f
fsuv1=[]


ptmp=np.linspace(0.001,0.999,5000)
ptmp1=np.append([0],ptmp)
ptmp1=np.append(ptmp1,[1])

for i in notebook.trange(ntest):
    ##cdf estimate by f
    ficdf= sess.run(yq_ ,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                q_:ptmp[:,None]
                                }).ravel()    
    ficdf=np.append([0],ficdf)
    ficdf=np.append(ficdf,12)

    fcdfest=interp1d(ficdf,ptmp1)
    fsuv1.append(1.-fcdfest(1))   
   

In [ ]:
#np.save('gfwidth_est',np.c_[width_est,fwidth_est])

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(tsuv1,gsuv1,'.')
plt.plot(tsuv1,fsuv1,'.')

plt.plot([0,1],[0,1])

#### Test the recovery of cdf

In [ ]:
#generate sample
np.random.seed(3421)
samps=np.random.choice(len(test_x),3)
#the mean and sd for the random sample
xtmp=np.linspace(0,7,50000)
qtmp=np.linspace(0,1,5000)

In [ ]:
plt.figure(figsize=(20,4))
plt.subplot(131)

plt.subplot(1,3,1)
i=samps[0]
tcdf=stats.weibull_min.cdf(x=xtmp,c=test_x[i,0],scale=test_x[i,1])
cdf=sess.run(gq ,feed_dict={x_:np.tile(test_x[i,:],(50000,1)),
                                yq_:xtmp[:,None]
                                }).ravel()
fcdf=sess.run(yq_,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                q_:qtmp[:,None]
                                }).ravel()
cngcdf=cdf
cnfcdf=fcdf

plt.plot(xtmp,tcdf)
plt.plot(xtmp,cdf)
plt.plot(fcdf,qtmp)


plt.subplot(1,3,2)
i=samps[1]
tcdf=stats.weibull_min.cdf(x=xtmp,c=test_x[i,0],scale=test_x[i,1])
cdf=sess.run(gq ,feed_dict={x_:np.tile(test_x[i,:],(50000,1)),
                                yq_:xtmp[:,None]
                                }).ravel()
fcdf=sess.run(yq_,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                q_:qtmp[:,None]
                                }).ravel()
cngcdf=np.c_[cngcdf,cdf]
cnfcdf=np.c_[cnfcdf,fcdf]

plt.plot(xtmp,tcdf)
plt.plot(xtmp,cdf)
plt.plot(fcdf,qtmp)

plt.subplot(1,3,3)
i=samps[2]
tcdf=stats.weibull_min.cdf(x=xtmp,c=test_x[i,0],scale=test_x[i,1])
cdf=sess.run(gq ,feed_dict={x_:np.tile(test_x[i,:],(50000,1)),
                                yq_:xtmp[:,None]
                                }).ravel()
fcdf=sess.run(yq_,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                q_:qtmp[:,None]
                                }).ravel()
cngcdf=np.c_[cngcdf,cdf]
cnfcdf=np.c_[cnfcdf,fcdf]

plt.plot(xtmp,tcdf)
plt.plot(xtmp,cdf)
plt.plot(fcdf,qtmp)

# np.save('cngcdf',cngcdf)
# np.save('cnfcdf',cnfcdf)

### Ten replications to evaluate the hard metrics

In [ ]:
##function to create replication
def rep_iter(x,y,frac=0.3):
    n=len(x)
    ntest=int(np.floor(frac*n))
    allidx=np.random.permutation(n)
    trainidx= allidx[ntest:]
    testidx= allidx[:ntest]
    return x[trainidx],y[trainidx],x[testidx],y[testidx]
    

In [ ]:
#initialize the metric

#trueth
tll=[]
tcal=[]
t90=[]
tmae=[]

#g
gll=[]
gcal=[]
g90=[]
gmae=[]

#f
fll=[]
fcal=[]
f90=[]
fmae=[]


In [ ]:
np.random.seed(2021)
for a in range(10):
    train_x,train_y,test_x,test_y=rep_iter(x,y)
    ntrain=len(train_x)
    ntest=len(test_x)

    

    sess = tf.compat.v1.Session()
    sess.run(initialize_all)
    gl=[]
    fl=[]


    for k in notebook.tnrange(pre_iter):
        i=np.random.choice(ntrain,batch_size,replace=False)
        ytmp = train_y[i,:]
        xtmp= train_x[i,:]
        #when we do not have f initially, we use a uniform distribution to extract points from support
        pre_ytmp=np.random.uniform(-1,14,(batch_size,1))  
        sess.run(pre_step,feed_dict={y_: ytmp,
                                    x_:xtmp,
                                    pre_y:pre_ytmp})


    for t in notebook.tnrange(iters):
        #randomly generate a minibatch from valid set
        i=np.random.choice(ntrain,batch_size,replace=False)
        qtmp=np.random.uniform(0, 1.0, [batch_size, 1])
        ytmp = train_y[i,:]
        xtmp= train_x[i,:]

        for i in range(0,1):
            _,glt=sess.run([g_step,g_loss], feed_dict={y_: ytmp,
                                    x_:xtmp,
                                    q_:qtmp})
        gl.append(glt)

        for i in range(0,2):
            _,flt=sess.run([f_step,f_loss],feed_dict={y_: ytmp,
                                    x_:xtmp,
                                    q_:qtmp})
        fl.append(flt)



    #####calculate metrics##############

    per=np.linspace(0.02,0.98,8) #quantile to study calibration

    calgt=np.empty([len(per)]) #tuecal
    for i in range(len(per)):
        ltmp=stats.weibull_min.ppf(0.5-per[i]/2.,c=test_x[:,0],scale=test_x[:,1])
        rtmp=stats.weibull_min.ppf(0.5+per[i]/2.,c=test_x[:,0],scale=test_x[:,1])
        calgt[i]=np.mean((ltmp<test_y.ravel())*(test_y.ravel()<rtmp))
                                                
    tcal.append(np.abs(calgt-per).mean()) #true calibration
    
    l90=stats.weibull_min.ppf(0.5-0.9/2.,c=test_x[:,0],scale=test_x[:,1])
    r90=stats.weibull_min.ppf(0.5+0.9/2.,c=test_x[:,0],scale=test_x[:,1])

    t90.append(np.mean((l90<test_y.ravel())*(test_y.ravel()<r90))) #true90 coverage
    #tmae.append(np.abs(test_y-stats.weibull_min.ppf(0.5,c=test_x[:,0],scale=test_x[:,1]).reshape(-1,1)).mean()) #true mae


    #lower and upper bound
    low=np.quantile(test_y,0.05)
    high=np.quantile(test_y,0.95)
    itv=np.linspace(low,high,9)
    itv=np.append(-np.infty,itv)
    itv=np.append(itv,np.infty)
    #outcome1 belongs to which interval
    id=np.zeros(ntest)
    for i in range(10):
        id=id+1*(test_y.ravel()>itv[i+1])
    id=id.astype('int')


    # estimation by g
    med_est=np.array([])
    ll_est=np.empty(ntest)
    cal_est=np.zeros_like(per)
    cover_90=0

    # estimation by f
    fmed_est=np.array([])
    fll_est=np.empty(ntest)
    fcal_est=np.zeros_like(per)
    fcover_90=0



    #use interpolation to recover cdf
    xtmp=np.linspace(-1,12,5000)
    ptmp=np.linspace(0.001,0.999,5000)
    ptmp1=np.append([0],ptmp)
    ptmp1=np.append(ptmp1,[1])





    #calculate the likelihood for ground truth
    loglikgth=np.zeros([ntest])
    for i in range(ntest):
        l=itv[id[i]]
        r=itv[id[i]+1]
        loglikgth[i]=np.log(stats.weibull_min.cdf(r,c=test_x[i,0],scale=test_x[i,1])-stats.weibull_min.cdf(l,c=test_x[i,0],scale=test_x[i,1])+1.e-10)


        #cdf estimate by g
        cdf=sess.run(gq ,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                    yq_:xtmp[:,None]
                                    }).ravel()

        cdf[0]=0
        cdf[-1]=1
        invcdfest=interp1d(cdf,xtmp)
        cdfest=interp1d(xtmp,cdf)


        ##cdf estimate by f
        ficdf= sess.run(yq_ ,feed_dict={x_:np.tile(test_x[i,:],(5000,1)),
                                    q_:ptmp[:,None]
                                    }).ravel()    
        ficdf=np.append([-1],ficdf)
        ficdf=np.append(ficdf,12)

        finvcdfest=interp1d(ptmp1,ficdf)
        fcdfest=interp1d(ficdf,ptmp1)

        #estimate the mae
        med_est=np.append(med_est,invcdfest(0.5))    
        fmed_est=np.append(fmed_est,finvcdfest(0.5))   


        #estimate the loglikelihood
        l=itv[id[i]]
        r=itv[id[i]+1]
        if(r==np.inf):
            ll_est[i]=np.log(1.-cdfest(l)+1.e-10)
            fll_est[i]=np.log(1.-fcdfest(l)+1.e-10)
        elif(l==-np.inf):
            ll_est[i]=np.log(cdfest(r)+1.e-10)
            fll_est[i]=np.log(fcdfest(r)+1.e-10)
        else:
            ll_est[i]=np.log(cdfest(r)-cdfest(l)+1.e-10)
            fll_est[i]=np.log(fcdfest(r)-fcdfest(l)+1.e-10)


        #estimate the calibration
        cal_est=cal_est+1.*(test_y[i]<invcdfest(0.5+per/2))*(test_y[i]>invcdfest(0.5-per/2))
        fcal_est=fcal_est+1.*(test_y[i]<finvcdfest(0.5+per/2))*(test_y[i]>finvcdfest(0.5-per/2))


        #estimate 90 coverage
        r=invcdfest(0.95)
        l=invcdfest(0.05)
        cover_90+=(test_y[i]<r)*(test_y[i]>l)

        fr=finvcdfest(0.95)
        fl=finvcdfest(0.05)
        fcover_90+=(test_y[i]<fr)*(test_y[i]>fl)

    #summary    
    tll.append(loglikgth.mean())    #true log likelihood

    cal_est=cal_est/ntest
    fcal_est=fcal_est/ntest

    #cal
    gcal.append(np.abs(cal_est-per).mean())
    fcal.append(np.abs(fcal_est-per).mean())

    #ll
    gll.append(ll_est.mean())
    fll.append(fll_est.mean())

    #90coverage
    g90.append(cover_90/ntest)
    f90.append(fcover_90/ntest)

    #mae
    gmae.append(np.abs(stats.weibull_min.ppf(0.5,c=test_x[:,0],scale=test_x[:,1])-med_est).mean())
    fmae.append(np.abs(stats.weibull_min.ppf(0.5,c=test_x[:,0],scale=test_x[:,1])-fmed_est).mean())

    

In [ ]:
def musd(x):
    print(np.mean(x),np.std(x))


In [ ]:
musd(tll)
musd(tcal)
musd(t90)
musd(tmae)
musd(gll)
musd(gcal)
musd(g90)
musd(gmae)
musd(fll)
musd(fcal)
musd(f90)
musd(fmae)
